# Space

In [ ]:
import os
import logging
import pandas as pd 
from IPython.display import display, HTML
KEY = 'WorkSpace'
WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY
# print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
import sys
from proj_space import PROJECT, TaskName, SPACE
sys.path.append(SPACE['CODE_FN'])
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
recfldtkn_config_path = os.path.join(SPACE['CODE_RFT'], 'config_recfldtkn')

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')


# Args

In [ ]:
###########################
RecName = 'TODO'# <-------- select your yaml file name
###########################

In [ ]:
from recfldtkn.configfn import load_cohort_args
from recfldtkn.configfn import load_record_args

cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)
RootIDLength = cohort_args['RootIDLength']
RootID = cohort_args['RootID']

rec_folder = cohort_args['rec_folder']
RawRootID = cohort_args['RawRootID'] 
record_args = load_record_args(RecName, cohort_args)

## [Updating Yaml File]

In [ ]:

# Create a HTML link and display it
path = record_args['yaml_file_path']
full_path = os.path.join(WORKSPACE_PATH, path)
display(HTML(f'{path} <a href="{full_path}" target="_blank">Open File</a>'))

## [Select A Cohort]

In [ ]:
import argparse
my_parser = argparse.ArgumentParser(description='Process Input.')

# Add the arguments
my_parser.add_argument('--cohort_name',
                    metavar='cohort_name',
                    default = None, 
                    type=str,
                    help='the cohort_name to process')

my_parser.add_argument('--cohort_label',
                    metavar='cohort_label',
                    default = None, 
                    type=str,
                    help='the label for cohort_name to process')

################### in notebook ###################
args_information = ['--cohort_label', '1']
args = my_parser.parse_args(args_information)
###################################################

cohort_label = int(args.cohort_label)
cohort_config = [v for k, v in cohort_args['CohortInfo'].items() if v['cohort_label'] == cohort_label][0]
cohort_name = cohort_config['cohort_name']
print('\n========== cohort_config ==========')
# print(cohort_config)
print(cohort_label, cohort_name)

# df_Human, df_Prt

In [ ]:
from recfldtkn.loadtools import filter_with_cohort_label, load_ds_rec_and_info
from recfldtkn.pipeline_record import get_parent_record_information

cohort_label_list = [cohort_label]
ds_Human, _ = load_ds_rec_and_info(cohort_args['RecName'], cohort_args, cohort_label_list = cohort_label_list)
df_Human = ds_Human.to_pandas()
prt_record_args, df_Prt, df_Human = get_parent_record_information(record_args, cohort_args, df_Human)
df_Prt = filter_with_cohort_label(df_Prt, cohort_label, cohort_args)
df_Human = df_Human[df_Human[RootID].isin(df_Prt[RootID].to_list())].reset_index(drop = True)
record_args['df_Prt'] = df_Prt
record_args['prt_record_args'] = prt_record_args

# OneCohortRec_args

In [ ]:
source_path_not_existence_flag = 0
OneCohortRec_args = record_args['CohortInfo'][cohort_name]
OneCohortRec_args['cohort_name'] = cohort_name  
OneCohortRec_args['cohort_label'] = cohort_label

for tablename, tableinfo in OneCohortRec_args.items():
    if tablename in ['cohort_name', 'cohort_label']: continue
    filename = tableinfo['raw_data_path']
    print(filename)
    if not os.path.exists(filename): source_path_not_existence_flag += 1

if source_path_not_existence_flag > 0:
    print(f'=== source_path_not_existence_flag: {source_path_not_existence_flag}')

OneCohortRec_args = record_args['CohortInfo'][cohort_name]
print('\n========== OneCohortRec_args ==========')
print(OneCohortRec_args)

#  HumanRecRaw

In [ ]:
print([tablename for tablename in OneCohortRec_args])

In [ ]:
from recfldtkn.pipeline_record import get_df_HumanSelected_from_OneCohortRecArgs
from recfldtkn.pipeline_record import get_HumanRawRec_for_HumanGroup

df_HumanSelected = get_df_HumanSelected_from_OneCohortRecArgs(df_Human, record_args, OneCohortRec_args, cohort_args)
logger.info(f'{df_HumanSelected.shape} === df_HumanSelected <-- df_Human: selected from in CohortLabel {cohort_label}: {cohort_name} and with RecordNum > 0')

######################
filepath_to_rawdf = {}
######################

for index_group, df_HumanGroup in df_HumanSelected.groupby('index_group'): 
    logger.info(f'current index_group: {index_group} ...')

    # ---------------------- this is the core part of the pipeline ----------------------
    # 7.1 get the df_HumanRawRec
    #     this function can be used independently to get the raw df_HumanRawRec. 
    df_HumanRawRec = get_HumanRawRec_for_HumanGroup(df_HumanGroup, OneCohortRec_args, RawRootID, filepath_to_rawdf)
    index = df_HumanRawRec[RawRootID].isin(df_HumanSelected[RawRootID].to_list())
    df_HumanRawRec = df_HumanRawRec[index].reset_index(drop = True)
    logger.info(f'current df_HumanRawRec: {df_HumanRawRec.shape} ...')

    break

# HumanRecFld

In [ ]:
# Create a HTML link and display it
path = record_args['pypath']
full_path = os.path.join(WORKSPACE_PATH, path)
link = f'{path} <a href="{full_path}" target="_blank">Open File</a>'
display(HTML(link))

## RawRec_to_RecAttr Code

In [ ]:
#-------------------
df = df_HumanRawRec

# -. filter out the records we don't need (optional) 

# -. create a new column for raw record id (optional)

# -. have a check that the raw record id is unique

# -. update datetime columns
   
# -. select a DT. TODO: you might need to localize the datetime to local timezone. 

# -. merge with the parent record (a must except Human Records)

# -. sort the table by Parent IDs and DT

# -. create a new column for RecID

df = df 
#-------------------

## Pin Down and Attr Cols

In [ ]:
# Create a HTML link and display it
path = record_args['yaml_file_path']
full_path = os.path.join(WORKSPACE_PATH, path)
link = f'{path} <a href="{full_path}" target="_blank">Open File</a>'
display(HTML(link))

In [ ]:
attr_cols = [

###################################
# ---- TOADD ATTRIBUTE COLUMNS ----
###################################

]
for i in attr_cols: print('-', i)

df[attr_cols].head()

## RawRec_to_RecAttr_fn

In [ ]:
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables
import inspect

###########################
def RawRec_to_RecAttr_fn(df_HumanRawRec, df_Human, cohort_args, record_args, fld_cols):
    
    #-------------------
    df = df_HumanRawRec

    # -. filter out the records we don't need (optional) 

    # -. create a new column for raw record id (optional)

    # -. have a check that the raw record id is unique

    # -. update datetime columns
    
    # -. select a DT. TODO: you might need to localize the datetime to local timezone. 

    # -. merge with the parent record (a must except Human Records)

    # -. sort the table by Parent IDs and DT

    # -. create a new column for RecID

    df = df 
    #-------------------

    df_HumanRecFld = df[fld_cols].reset_index(drop = True)
    return df_HumanRecFld
###########################

RawRec_to_RecAttr_fn.fn_string = inspect.getsource(RawRec_to_RecAttr_fn)


In [ ]:
prefix = ['import pandas as pd', 'import numpy as np']
pycode = convert_variables_to_pystirng(fn_variables = [RawRec_to_RecAttr_fn], prefix = prefix)
RecName = record_args['RecName']
pypath = record_args['pypath']
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

RecName = record_args['RecName']
pypath = record_args['pypath']
module = load_module_variables(pypath)
RawRec_to_RecAttr_fn = module.MetaDict['RawRec_to_RecAttr_fn']

In [ ]:

df_HumanRecAttr = RawRec_to_RecAttr_fn(df_HumanRawRec, df_Human, cohort_args, record_args, attr_cols)
df_HumanRecAttr